In [ ]:
!pip install googletrans==4.0.0-rc1

In [ ]:
!pip install langid

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
import re
from googletrans import Translator
import langid
import ast
from sklearn.naive_bayes import MultinomialNB

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
data = pd.read_csv('smartphones_data.csv')

In [ ]:
data.head(5)

,Link,Mobile Name,Price,Rating,Comments,Seller Name,Positive Seller Ratings,Ship on Time,Chat Response Rate
0,https://www.daraz.pk/products/y02-t-4-gb-64-gb...,Vivo Y02 T RAM 4 GB ROM 64 GB Front Camera 5 M...,"Rs. 39,890",NaN,[],Sid Sam Communication Khanewal,93%,92%,98%
1,https://www.daraz.pk/products/6gb-6gb-y17s-128...,Vivo Y17s - 6GB + 6GB Expendable RAM 128 ROM -...,"Rs. 45,499",4.0,['good 💯😊'],Kashif Electronics (Karachi),89%,93%,100%
2,https://www.daraz.pk/products/a14-6gb-128-gb-p...,Samsung Galaxy A14 - 6GB ROM 128 GB - PTA Appr...,"Rs. 57,199",NaN,[],Mobile Arena (Karachi),89%,46%,91%
3,https://www.daraz.pk/products/10-4gb-128gb-hd-...,"Tecno Spark 10 - 4GB RAM 128GB ROM - 6.6"" HD+9...","Rs. 33,499",NaN,[],Kashif Electronics (Karachi),89%,93%,100%
4,https://www.daraz.pk/products/20-i-4-gb-128-8-...,Infinix Hot 20 i RAM 4 GB ROM 128 Front Camera...,"Rs. 42,000",NaN,[],Atta Communication Khanewal,94%,100%,100%


In [ ]:
print(data.shape)

(890, 9)


**Drop the rows which has no comments so we cant do sentimantal analysis on those products**

In [ ]:
# Define a function to safely evaluate the string representation of lists
def safe_eval(x):
    try:
        return ast.literal_eval(x) if pd.notna(x) else []
    except (SyntaxError, ValueError):
        return []

In [ ]:
# Convert string representations to actual lists using the safe_eval function
data['Comments'] = data['Comments'].apply(safe_eval)

# Drop rows with empty comments
data = data[data['Comments'].apply(lambda x: bool(x))]

# Save the modified DataFrame to a new CSV file
data.to_csv('df.csv', index=False)

In [ ]:
df = pd.read_csv('df.csv')

**Split the comments**

In [ ]:
#Split each comment into a list of individual comments
df['Comments'] = df['Comments'].apply(lambda x: x.strip("[]").replace("'", "").split(", "))

# Explode the list of comments into separate rows
df = df.explode('Comments')

# Reset the index
df.reset_index(drop=True, inplace=True)

# Save the modified dataset
df.to_csv('dataf.csv', index=False)

In [ ]:
# Drop rows with empty comments
df['Comments'] = df['Comments'].apply(lambda x: np.nan if x.isspace() or not x else x)
df.dropna(subset=['Comments'], inplace=True)

# Reset the index
df.reset_index(drop=True, inplace=True)

# Save the modified dataset
df.to_csv('final.csv', index=False)

In [ ]:
final = pd.read_csv('final.csv')

**Data Cleaning**

In [ ]:
!pip install contractions


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 10.2 MB/s eta 0:00:00


In [ ]:
pip install emoji

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 358.9/358.9 kB 5.8 MB/s eta 0:00:00


In [ ]:
import re
import string
import contractions
from nltk.corpus import stopwords
import emoji
from googletrans import Translator

In [ ]:
def to_lower(text):
    return text.lower()

In [ ]:
def remove_specialChar(text):

    text = re.sub('[^A-Za-z]+', ' ', text)
    return text

In [ ]:
def contraction_expansion(text):

    text = contractions.fix(text)
    return text

In [ ]:
def remove_stop_words(text):
    stop_words = set(stopwords.words('english'))
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_words)

In [ ]:
def remove_emoji(text):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F700-\U0001F77F"  # alchemical symbols
                               u"\U0001F780-\U0001F7FF"  # Geometric Shapes Extended
                               u"\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
                               u"\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
                               u"\U0001FA00-\U0001FA6F"  # Chess Symbols
                               u"\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
                               u"\U00002702-\U000027B0"  # Dingbats
                               u"\U000024C2-\U0001F251"
                               "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text)
    return text

In [ ]:
def translate_to_english(comment):
    translator = Translator()
    translated_comment = translator.translate(comment, dest='en').text
    return translated_comment



In [ ]:
def data_cleaning(text):

    text = to_lower(text)
    text = remove_specialChar(text)
    text = contraction_expansion(text)
    text = remove_stop_words(text)
    text = remove_emoji(text)
    text = translate_to_english(text)
    return text

In [ ]:
final['Comments'] = final['Comments'].astype(str)

In [ ]:
final = final[final['Comments'].str.len() > 0]

In [ ]:
final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1794 entries, 0 to 1819
Data columns (total 9 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Link                     1794 non-null   object 
 1   Mobile Name              1794 non-null   object 
 2   Price                    1794 non-null   object 
 3   Rating                   1794 non-null   float64
 4   Comments                 1794 non-null   object 
 5   Seller Name              1794 non-null   object 
 6   Positive Seller Ratings  1775 non-null   object 
 7   Ship on Time             1775 non-null   object 
 8   Chat Response Rate       1745 non-null   object 
dtypes: float64(1), object(8)
memory usage: 140.2+ KB


In [ ]:
final['translated_comments'] = final['Comments'].apply(lambda x: data_cleaning(x) if pd.notnull(x) else x)

In [ ]:
# when we remove emogis there are some comments who just have emogis so they show null, so we comapre with the lenght to remove them so it reduces the no of comments.

In [ ]:
print(final.shape)

(1794, 10)


In [ ]:
for i in final['translated_comments']:
  print(i)

In [ ]:
final['translated_comments'].to_csv('translated_comments.csv', index=False, header=['translated_comments'])

In [ ]:
final.to_csv("withScore_dataSet.csv", index=False)